### <u> Convert stellar spectrum to an audio spectrum and generate the sound</u>

First, we import relevant modules:

In [ ]:
%matplotlib inline
%reload_ext autoreload 
%autoreload 2
import matplotlib.pyplot as plt
import ffmpeg as ff
import wavio as wav
from strauss.sonification import Sonification
from strauss.sources import MultiEvents
from strauss import channels
from strauss.score import Score
import numpy as np
from strauss.generator import Sampler, Synthesizer
import IPython.display as ipd
from IPython.core.display import display
import os
from scipy.signal import savgol_filter

then, define display function for sonifications:

In [ ]:
def display_sonification(soni, color='C0'):
    time = soni.out_channels['0'].samples / soni.out_channels['0'].samprate

    bellenv = pow(time,0.25)*np.exp(-time/1.2)**1.
    

    for i in range(len(soni.out_channels)):
        plt.plot(time[::20], bellenv[::20]*soni.out_channels[str(i)].values[::20]-1.2*i, label=soni.channels.labels[i], c=color)

    plt.xlabel('Time (s)')
    plt.ylabel('Relative Amplitude')
    plt.legend(frameon=False, loc=5)
    plt.xlim(-time[-1]*0.05,time[-1]*1.2)
    for s in plt.gca().spines.values():
        s.set_visible(False)
    plt.gca().get_yaxis().set_visible(False)

    if len(soni.channels.labels) == 1:
        # we have used 441000 Hz everywhere above as standard, but to quickly hear the sonification sped up / slowed down,
        # you can modify the 'rate' argument below (e.g. multiply by 0.5 for half speed, by 2 for double speed, etc)
        outfmt = np.column_stack([soni.out_channels['0'].values, soni.out_channels['0'].values]).T
    else:
        outfmt = np.column_stack([soni.out_channels['0'].values, soni.out_channels['1'].values]).T
    display(ipd.Audio(outfmt*bellenv,rate=soni.out_channels['0'].samprate, autoplay=False))


In [ ]:
data = np.genfromtxt('../data/datasets/M_spectrum.txt')
Nharms = data.shape[0]
freq = 1/data[:,0]
freq = (freq - freq.min()) / (freq.max() - freq.min())
freq = ((freq)*8e3) + 20

chords = [freq]
length = "0m 1s"
score =  Score(chords, length)
system = 'stereo'

In [ ]:
generator = Synthesizer()

generator.load_preset('spectraliser')

In [ ]:
mapvals =  {'phi': lambda x : x,
            'theta': lambda x : x,
            'pitch' : lambda x: x,
            'volume' : lambda x: x}

maplims =  {'phi': (0, 360),
            'theta': (0, 180),
            'pitch' : ('0', '100'),
            'volume' : (0, 22)}

# set as high and low pitch respectively
ps = np.arange(Nharms)

# hard pan the sets left and right
phis = np.zeros(Nharms)

# place on equatorial plane
thetas = np.ones(Nharms)*90

In [ ]:
volumes_M = data[:,1]
volumes_smooth_M = savgol_filter(volumes_M, 51, 3)

volumes_negfeats_M = -np.clip((volumes_M - volumes_smooth_M), -np.inf, 0)
volumes_posfeats_M = np.clip((volumes_M - volumes_smooth_M), 0, np.inf)
plt.plot(freq, volumes_M, c='k', alpha=0.3)
plt.plot(freq, volumes_smooth_M - 0.6*volumes_M.max())
plt.plot(freq, volumes_negfeats_M**0.5 - 1.2*volumes_M.max())
plt.plot(freq, volumes_posfeats_M**0.5 - 1.8*volumes_M.max())
plt.gca().get_yaxis().set_visible(False)
plt.xlabel("Frequency (Hz)")
plt.savefig("../../M_class_spectrum.pdf")
plt.show()

In [ ]:
events = MultiEvents(['phi', 'theta', 'pitch', 'volume'])
events.raw_mapping = dict(zip(['phi', 'theta', 'pitch', 'volume'],
                           [phis, thetas, ps, volumes_M]))

events.apply_mapping_functions(mapvals, maplims)
events.n_sources = ps.shape[0]
soni_M = Sonification(score, events, generator, system)
soni_M.render()

In [ ]:
display_sonification(soni_M, color='0.8')

In [ ]:
events = MultiEvents(['phi', 'theta', 'pitch', 'volume'])
events.raw_mapping = dict(zip(['phi', 'theta', 'pitch', 'volume'],
                           [phis, thetas, ps, volumes_smooth_M]))

events.apply_mapping_functions(mapvals, maplims)
events.n_sources = ps.shape[0]
soni_M_cont = Sonification(score, events, generator, system)
soni_M_cont.render()

In [ ]:
display_sonification(soni_M_cont, color='C0')

In [ ]:
events = MultiEvents(['phi', 'theta', 'pitch', 'volume'])
events.raw_mapping = dict(zip(['phi', 'theta', 'pitch', 'volume'],
                           [phis, thetas, ps, volumes_negfeats_M]))

events.apply_mapping_functions(mapvals, maplims)
events.n_sources = ps.shape[0]
soni_M_neg = Sonification(score, events, generator, system)
soni_M_neg.render()

In [ ]:
display_sonification(soni_M_neg, color='C1')
soni_M_neg.save_stereo('../../M_class_cymbal.wav')

In [ ]:
events = MultiEvents(['phi', 'theta', 'pitch', 'volume'])
events.raw_mapping = dict(zip(['phi', 'theta', 'pitch', 'volume'],
                           [phis, thetas, ps, volumes_posfeats_M]))

events.apply_mapping_functions(mapvals, maplims)
events.n_sources = ps.shape[0]
soni_M_pos = Sonification(score, events, generator, system)
soni_M_pos.render()

In [ ]:
display_sonification(soni_M_pos, color='C2')

Run `soni.save_stereo('<directory/to/filename.wav>')` if you want to save the sonification to a file.


In [ ]:
data = np.genfromtxt('../data/datasets/O_spectrum.txt')
volumes_O = data[:,1]
volumes_smooth_O = savgol_filter(volumes_O, 51, 3)

volumes_negfeats_O = -np.clip((volumes_O - volumes_smooth_O), -np.inf, 0)#/volumes_smooth_O
volumes_posfeats_O = np.clip((volumes_O - volumes_smooth_O), 0, np.inf)#/volumes_smooth_O

plt.plot(freq, volumes_O, c='k', alpha=0.3)
plt.plot(freq, volumes_smooth_O - 0.6*volumes_O.max())
plt.plot(freq, volumes_negfeats_O - 1.2*volumes_O.max())
plt.plot(freq, volumes_posfeats_O - 1.8*volumes_O.max())
plt.show()

In [ ]:
events = MultiEvents(['phi', 'theta', 'pitch', 'volume'])
events.raw_mapping = dict(zip(['phi', 'theta', 'pitch', 'volume'],
                           [phis, thetas, ps, volumes_O]))

events.apply_mapping_functions(mapvals, maplims)
events.n_sources = ps.shape[0]
soni_O = Sonification(score, events, generator, system)
soni_O.render()

In [ ]:
display_sonification(soni_O, color='0.8')

In [ ]:
events = MultiEvents(['phi', 'theta', 'pitch', 'volume'])
events.raw_mapping = dict(zip(['phi', 'theta', 'pitch', 'volume'],
                           [phis, thetas, ps, volumes_smooth_O]))

events.apply_mapping_functions(mapvals, maplims)
events.n_sources = ps.shape[0]
soni_O_cont = Sonification(score, events, generator, system)
soni_O_cont.render()

In [ ]:
display_sonification(soni_O_cont, color='C0')

In [ ]:
events = MultiEvents(['phi', 'theta', 'pitch', 'volume'])
events.raw_mapping = dict(zip(['phi', 'theta', 'pitch', 'volume'],
                           [phis, thetas, ps, volumes_negfeats_O]))

events.apply_mapping_functions(mapvals, maplims)
events.n_sources = ps.shape[0]
soni_O_neg = Sonification(score, events, generator, system)
soni_O_neg.render()

In [ ]:
display_sonification(soni_O_neg, color='C1')

In [ ]:
events = MultiEvents(['phi', 'theta', 'pitch', 'volume'])
events.raw_mapping = dict(zip(['phi', 'theta', 'pitch', 'volume'],
                           [phis, thetas, ps, volumes_posfeats_O]))

events.apply_mapping_functions(mapvals, maplims)
events.n_sources = ps.shape[0]
soni_O_pos = Sonification(score, events, generator, system)
soni_O_pos.render()

In [ ]:
display_sonification(soni_O_pos, color='C2')